In [ ]:
# pytorch mlp for multiclass classification
from numpy import vstack
from numpy import argmax
from pandas import read_csv
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import sys
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import pandas as pd
from torchvision import transforms
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss
import numpy as np
import seaborn as sns
import tensorflow as tf
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk import word_tokenize
import torch
from torch import nn
import scipy
from torch import optim
import time
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

def MyCleanText(X, 
               lowercase=False, # mettre en minuscule
               removestopwords=False, # supprimer les stopwords
               removedigit=False, # supprimer les nombres  
               getstemmer=False, # conserver la racine des termes
               getlemmatisation=False # lematisation des termes 
              ):
    
    sentence=str(X)

    # suppression des caractères spéciaux
    sentence = re.sub(r'[^\w\s]',' ', sentence)
    # suppression de tous les caractères uniques
    sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', sentence)
    # substitution des espaces multiples par un seul espace
    sentence = re.sub(r'\s+', ' ', sentence, flags=re.I)

    # decoupage en mots
    tokens = word_tokenize(sentence)
    if lowercase:
          tokens = [token.lower() for token in tokens]

    # suppression ponctuation
    table = str.maketrans('', '', string.punctuation)
    words = [token.translate(table) for token in tokens]

    # suppression des tokens non alphabetique ou numerique
    words = [word for word in words if word.isalnum()]
    
    # suppression des tokens numerique
    if removedigit:
        words = [word for word in words if not word.isdigit()]

    # suppression des stopwords
    if removestopwords:
        words = [word for word in words if not word in stop_words]

    # lemmatisation
    if getlemmatisation:
        lemmatizer=WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word)for word in words]
        

    # racinisation
    if getstemmer:
        ps = PorterStemmer()
        words=[ps.stem(word) for word in words]
        
    sentence= ' '.join(words)
  
    return sentence   

from sklearn.base import BaseEstimator, TransformerMixin

class TextNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 removestopwords=False, # suppression des stopwords
                 lowercase=False,# passage en minuscule
                 removedigit=False, # supprimer les nombres  
                 getstemmer=False,# racinisation des termes 
                 getlemmatisation=False # lemmatisation des termes  
                ):
        
        self.lowercase=lowercase
        self.getstemmer=getstemmer
        self.removestopwords=removestopwords
        self.getlemmatisation=getlemmatisation
        self.removedigit=removedigit
        #self.transform = transforms.Compose([transforms.ToTensor()])  

    def transform(self, X, **transform_params):
        # Nettoyage du texte
        X=X.copy() # pour conserver le fichier d'origine
        return [MyCleanText(text,lowercase=self.lowercase,
                            getstemmer=self.getstemmer,
                            removestopwords=self.removestopwords,
                            getlemmatisation=self.getlemmatisation,
                            removedigit=self.removedigit) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self
    
    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

    def get_params(self, deep=True):
        return {
            'lowercase':self.lowercase,
            'getstemmer':self.getstemmer,
            'removestopwords':self.removestopwords,
            'getlemmatisation':self.getlemmatisation,
            'removedigit':self.removedigit
        }    
    
    def set_params (self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter,value)
        return self    
        
        pipe = Pipeline([("cleaner", TextNormalizer(lowercase=True)),
                 ("tfidf_vectorizer", TfidfVectorizer(max_features=NB_FEATURES))])

        #pipe.fit(self.X)
        #self.X=pipe.transform(self.X)
        self.X=pipe.fit_transform(self.X)
        # sauvegarde du vocabulaire du TF_IDF pour tester d'autres données
        print (pipe.named_steps['tfidfvectorizer'].vocabulary)
        #corpus  = [tokenize(doc) for doc in self.X]
        #lexicon = gensim.corpora.Dictionary(corpus)
        #lexicon.save_as_text('lexicon.txt', sort_by_word=True)
        pickle.dump(pipe.named_steps['tfidfvectorizer'].vocabulary,open("feature.pkl","wb"))

        self.X=self.X.toarray()
        self.X = self.X.astype('float32')
        self.y = LabelEncoder().fit_transform(self.y)


# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

my_local_drive='/content/gdrive/MyDrive/Colab Notebooks/PP2 Mio'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/PP2 Mio


'/content/gdrive/MyDrive/Colab Notebooks/PP2 Mio'

In [ ]:

NB_FEATURES=1000
EPOCHS=150



# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None, sep='\t', encoding='utf8')
        #print(df.head())
        # store the inputs and outputs
        self.X = df.values[:, :-1]

        self.y = df.values[:, -1]

        # création du pipeline
        pipe = Pipeline([("cleaner", TextNormalizer(lowercase=True)),
                 ("tfidf_vectorizer", TfidfVectorizer(max_features=NB_FEATURES))])


        self.X=pipe.fit_transform(self.X)
        # sauvegarde du vocabulaire du TF_IDF pour tester d'autres données
        pickle.dump(pipe.named_steps['tfidf_vectorizer'].vocabulary_,open("vocabulaire.txt","wb"))

        # attention important il faut convertir le résultat de TF_IDF en array et dire que dedans ce 
        # sont des floats
        self.X=self.X.toarray()
        self.X = self.X.astype('float32')
        # ne pas oublier de transformer le y via label encoder -> chaque classe aura un numéro
        self.y = LabelEncoder().fit_transform(self.y)


    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
       # return self.transform(self.x_data[idx]), self.transform(self.y_data[idx])
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

    

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        #self.hidden3 = Linear(8, 3)
        self.hidden3 = Linear(8, 4)
        xavier_uniform_(self.hidden3.weight)
        #self.act3 = Softmax(dim=1)
        self.act3 = Softmax(dim=1)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer
        X = self.hidden3(X)
        return torch.softmax(X, dim=1)
        X = self.act3(X)

        return X

# prepare the dataset
def prepare_data(path):
    # load the dataset
    print("----- dans prepare data avant CSV Dataset")
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    print ("Taille du jeu de test et d'apprentissage",len(train),len(test))
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

# train the model
def train_model(train_dl, model):
    # define the optimization
    #criterion = BCELoss()
    # attention ici c'est une cross entropy
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(EPOCHS):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            
            # clear the gradients
            optimizer.zero_grad()
       
            # compute the model output
            yhat = model(inputs)#.numpy()
            
            # calculate loss
            loss = criterion(yhat, targets)
            
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat


# prepare the data
start_time = time.time()
path = 'ReviewsLabelled2.csv'
train_dl, test_dl = prepare_data(path)

print("Len train_dl = nb batchsize", len(train_dl), len(train_dl.dataset), len(test_dl.dataset),"\n")
print("Prepare the data : %s seconds \n" % (time.time() - start_time))
# define the network
model = MLP(NB_FEATURES)
start_time = time.time()
# train the model
print(len(train_dl.dataset), len(test_dl.dataset))
#train_dl.describe(include = [object])
train_model(train_dl, model)
print("Train : %s seconds \n" % (time.time() - start_time))

start_time = time.time()
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)
print("Evaluate : %s seconds \n" % (time.time() - start_time))
# Creation du modele general sur toutes les donnée pour faire de la prédiction

# NORMALEMENT ICI IL FAUT RELANCER L'APPRENTISSAGE SUR TOUT LE JEU DE DONNEES POUR AVOIR UN BON MODELE


# EN DESSOUS C'EST POUR MONTRER COMMENT ON PEUT UTILISER LE MODELE. JE N'AI PAS TROP TESTE. 
# J'AI JUSTE VERIFIE QUE LES PRETRAIREMENTS ETAIENT BIEN FAIT : on nettoie et on fait le tf-idf avec le vocabulaire
# sur lequel on a appris
clean_data=pickle.load( open( "vocabulaire.txt", "rb" ) )


#je prepare The row que je vais lui passer
#text = ["Good case, Excellent value"]
#text_normalizer=TextNormalizer(lowercase=True,getstemmer=True,removestopwords=True,getlemmatisation=True,removedigit=True) 
#text=text_normalizer.fit_transform(text)
#tfidf=TfidfVectorizer(vocabulary=clean_data)
#text=tfidf.fit_transform(text).toarray()
#print(text)
#yhat = predict(row, model)
#print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

#df = read_csv(path, header=None, sep='\t', encoding='utf8')

row = ['There is not a deal good enough that would drag me into that establishment again.']  #yeld
#df.values[2000:2006, :-1]

row=TextNormalizer(lowercase=True).fit_transform(row)

print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
row = row.astype('float32')
for i in range (0, len(row)):
  yhat = predict(row[0], model)
  print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

row = ["Overall, I would recommend this phone over the new Walkman."] # Amazon
#df.values[2000:2006, :-1]

row=TextNormalizer(lowercase=True).fit_transform(row)

print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
row = row.astype('float32')
for i in range (0, len(row)):
  yhat = predict(row[0], model)
  print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

row = ["I purchased this and within 2 days it was no longer working!!!!!!!!!"] # Amazon
#df.values[2000:2006, :-1]

row=TextNormalizer(lowercase=True).fit_transform(row)

print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
row = row.astype('float32')
for i in range (0, len(row)):
  yhat = predict(row[0], model)
  print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))


row = ["I think the most wonderful parts (literally, full of ""wonder"") are the excerpts from his works.  "] #imdb
#df.values[2000:2006, :-1]

row=TextNormalizer(lowercase=True).fit_transform(row)

print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
row = row.astype('float32')
for i in range (0, len(row)):
  yhat = predict(row[0], model)
  print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))




----- dans prepare data avant CSV Dataset
Taille du jeu de test et d'apprentissage 2011 990
Len train_dl = nb batchsize 63 2011 990
Prepare the data : 1.5746726989746094 seconds 

2011 990
Train : 12.264191150665283 seconds 

Accuracy: 0.800
Evaluate : 0.008051156997680664 seconds 

['there is not deal good enough that would drag me into that establishment again'] 

Predicted: [[2.7477548e-02 1.4466151e-04 2.2956135e-06 9.7237557e-01]] (class=3)
['overall would recommend this phone over the new walkman'] 

Predicted: [[1.0000000e+00 7.6456796e-09 3.5772047e-08 8.1461762e-09]] (class=0)
['i purchased this and within 2 days it was no longer working'] 

Predicted: [[9.9355054e-01 1.7775620e-03 1.0342069e-05 4.6616294e-03]] (class=0)
['i think the most wonderful parts literally full of wonder are the excerpts from his works'] 

Predicted: [[1.8176020e-04 9.9981815e-01 2.9544825e-08 6.6157739e-08]] (class=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:154: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)


In [ ]:
# make a single prediction
row = ['There is not a deal good enough that would drag me into that establishment again.' , "The food sucked, which we expected but it sucked more than we could have imagined." ] # Yeld
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

# make a single prediction
row = ["The mic is great."] #, 'I plugged it in only to find out not a darn thing worked.'] # Amazon
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

row = ['There is not a deal good enough that would drag me into that establishment again.'] # , "The food sucked, which we expected but it sucked more than we could have imagined." ] # Yeld
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

# make a single prediction
row = ["Overall, I would recommend this phone over the new Walkman.", 'This is infuriating.'] # Amazon
row=TextNormalizer(lowercase=True).fit_transform(row)
#row=clean_data.named_steps["cleaner"].fit_transform(row)
print (row, '\n')
vectorizer2 = TfidfVectorizer(vocabulary=clean_data)

row=vectorizer2.fit_transform(row).toarray()
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))


['there is not deal good enough that would drag me into that establishment again', 'the food sucked which we expected but it sucked more than we could have imagined'] 

Predicted: [[[8.7326491e-01 9.9956721e-01 7.1431398e-01 2.5923661e-04]
  [1.2673506e-01 4.3276491e-04 2.8568596e-01 9.9974078e-01]]] (class=7)
['the mic is great'] 

Predicted: [[[1. 1. 1. 1.]]] (class=0)
['there is not deal good enough that would drag me into that establishment again'] 

Predicted: [[[1. 1. 1. 1.]]] (class=0)
['overall would recommend this phone over the new walkman', 'this is infuriating'] 

Predicted: [[[9.99882340e-01 3.59662389e-03 9.60669160e-01 1.15713347e-02]
  [1.17684205e-04 9.96403337e-01 3.93308140e-02 9.88428652e-01]]] (class=0)
